In [ ]:
import paramiko
from IPython.core.magic import register_cell_magic

def load_connection_config(config_file='connection_config.txt'):
    """Load connection configuration from file"""
    config = {}
    with open(config_file, 'r') as f:
        for line in f:
            line = line.strip()
            if '=' in line and not line.startswith('#'):
                key, value = line.split('=', 1)
                key = key.strip()
                value = value.strip()
                # Convert port to integer
                if key == 'port':
                    value = int(value)
                config[key] = value
    return config

# Load configuration
try:
    secrets = load_connection_config('connection_config.txt')
    hostname = secrets['hostname']
    port = secrets['port']
    username = secrets['username']
    password = secrets['password']
    
    # Test connection
    print(f"Connecting to {hostname}:{port} as {username}...")
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname=hostname, port=port, username=username, password=password, timeout=10)
    print("✓ Connection successful!\n")
    client.close()
except FileNotFoundError:
    print("⚠ Warning: connection_config.txt not found. Please create it first.")
    hostname = port = username = password = None
except Exception as e:
    print(f"✗ Connection failed: {e}\n")
    hostname = port = username = password = None

# Unified VMS command
@register_cell_magic
def vms(line, cell):
    """
    Unified VMS command - handles shell, Python, and file operations
    
    Usage:
        %%vms                                      # Execute shell commands
        %%vms python                               # Execute Python (uses venv if available)
        %%vms python:ml_env                        # Execute Python with specific venv
        %%vms python persistent script.py          # Append to file and execute
        %%vms python:ml_env persistent script.py   # Append to file and execute in venv
    """
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname=hostname, port=port, username=username, password=password, timeout=10)
    
    line = line.strip()
    
    # Mode 1: Shell commands (default)
    if not line or not line.startswith('python'):
        stdin, stdout, stderr = client.exec_command(cell)
        output = stdout.read().decode()
        errors = stderr.read().decode()
        
        client.close()
        
        if errors:
            print("STDERR:", errors)
        if output:
            print(output)
        return
    
    # Parse Python modes
    venv_name = None
    persistent = False
    filename = 'persistent.py'
    
    # Check if specific venv is specified (python:venv_name)
    if ':' in line:
        prefix, rest = line.split(':', 1)
        rest_parts = rest.strip().split()
        venv_name = rest_parts[0]
        
        # Check for persistent mode
        if len(rest_parts) > 1 and rest_parts[1] == 'persistent':
            persistent = True
            if len(rest_parts) > 2:
                filename = rest_parts[2]
    else:
        # No specific venv, check for persistent mode
        parts = line.split()
        if len(parts) > 1 and parts[1] == 'persistent':
            persistent = True
            if len(parts) > 2:
                filename = parts[2]
    
    # Determine which Python to use
    if venv_name:
        # Use specified venv
        python_cmd = f'{venv_name}/bin/python3'
    else:
        # Auto-detect default venv
        default_venv = 'ml_env'
        stdin, stdout, stderr = client.exec_command(f'test -f {default_venv}/bin/python3 && echo "yes" || echo "no"')
        venv_exists = stdout.read().decode().strip() == "yes"
        python_cmd = f'{default_venv}/bin/python3' if venv_exists else 'python3'
    
    # Mode 2 & 3: Python execution (non-persistent)
    if not persistent:
        command = f'{python_cmd} << EOF\n{cell}\nEOF'
        stdin, stdout, stderr = client.exec_command(command)
        output = stdout.read().decode()
        errors = stderr.read().decode()
        
        client.close()
        
        if errors:
            print("STDERR:", errors)
        if output:
            print(output)
        return
    
    # Mode 4 & 5: Persistent Python execution
    command = f'cat >> {filename} << EOF\n{cell}\nEOF\n{python_cmd} {filename}'
    stdin, stdout, stderr = client.exec_command(command)
    output = stdout.read().decode()
    errors = stderr.read().decode()
    
    client.close()
    
    if errors:
        print("STDERR:", errors)
    if output:
        print(output)

def setup_venv(venv_name='ml_env', packages=None, force_reinstall=False):
    """
    Set up a Python virtual environment on remote machine with ML packages
    
    Args:
        venv_name: Name of the virtual environment (default: 'ml_env')
        packages: List of packages to install (default: numpy, pandas, matplotlib, scikit-learn, fastai, tinygrad)
        force_reinstall: If True, removes existing venv and creates fresh one
    """
    if packages is None:
        packages = ['numpy', 'pandas', 'matplotlib', 'scikit-learn', 'fastai', 'tinygrad']
    
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname=hostname, port=port, username=username, password=password, timeout=10)
    
    print(f"Setting up virtual environment: {venv_name}")
    print("=" * 60)
    
    # Step 1: Remove existing venv if force_reinstall
    if force_reinstall:
        print("\n1. Removing existing virtual environment...")
        stdin, stdout, stderr = client.exec_command(f'rm -rf {venv_name}')
        stdout.channel.recv_exit_status()
        print("   ✓ Cleaned up old environment")
    
    # Step 2: Check if venv exists
    print("\n2. Checking for existing virtual environment...")
    stdin, stdout, stderr = client.exec_command(f'test -d {venv_name} && echo "exists" || echo "not found"')
    exists = stdout.read().decode().strip()
    
    if exists == "not found":
        print(f"   Creating new virtual environment: {venv_name}")
        stdin, stdout, stderr = client.exec_command(f'python3 -m venv {venv_name}')
        stdout.channel.recv_exit_status()
        print("   ✓ Virtual environment created")
    else:
        print(f"   ✓ Virtual environment already exists: {venv_name}")
    
    # Step 3: Upgrade pip
    print("\n3. Upgrading pip...")
    stdin, stdout, stderr = client.exec_command(
        f'{venv_name}/bin/pip install --upgrade pip'
    )
    stdout.channel.recv_exit_status()
    print("   ✓ Pip upgraded")
    
    # Step 4: Install packages
    print("\n4. Installing packages...")
    packages_str = ' '.join(packages)
    print(f"   Installing: {packages_str}")
    
    stdin, stdout, stderr = client.exec_command(
        f'{venv_name}/bin/pip install {packages_str}'
    )
    
    # Stream output
    while True:
        line = stdout.readline()
        if not line:
            break
        print(f"   {line.rstrip()}")
    
    stdout.channel.recv_exit_status()
    
    # Step 5: Verify installation
    print("\n5. Verifying installation...")
    package_pattern = '|'.join(packages)
    stdin, stdout, stderr = client.exec_command(
        f'{venv_name}/bin/pip list | grep -E "{package_pattern}"'
    )
    installed = stdout.read().decode()
    print(f"\n   Installed packages:\n{installed}")
    
    client.close()
    
    print("\n" + "=" * 60)
    print("✓ Virtual environment setup complete!")
    print(f"\nUsage:")
    print(f"   %%vms python:{venv_name}")
    print(f"   %%vms python:{venv_name} persistent script.py")
    print("=" * 60)

if hostname:
    print("✓ VMS Magic command ready:")
    print("  - %%vms                                      : Execute shell commands")
    print("  - %%vms python                               : Execute Python (auto venv)")
    print("  - %%vms python:venv_name                     : Execute Python (specific venv)")
    print("  - %%vms python persistent file.py            : Persistent Python (auto venv)")
    print("  - %%vms python:venv_name persistent file.py  : Persistent Python (specific venv)")
    print("  - setup_venv(name, packages, force)          : Setup virtual environment")

Connecting to 2.tcp.ngrok.io:19261 as solvituser...


✓ Connection successful!

✓ VMS Magic command ready:
  - %%vms                                      : Execute shell commands
  - %%vms python                               : Execute Python (auto venv)
  - %%vms python:venv_name                     : Execute Python (specific venv)
  - %%vms python persistent file.py            : Persistent Python (auto venv)
  - %%vms python:venv_name persistent file.py  : Persistent Python (specific venv)
  - setup_venv(name, packages, force)          : Setup virtual environment


### Testing tinygrad with my local Intel GPU

This demonstrates training a neural network using tinygrad on a remote machine with Intel GPU. Pytorch does not work with Intel so 
Tinygrad was chosen to demonstrate:

**Steps:**
1. **Setup environment** - Create a virtual environment with tinygrad and dependencies (numpy, pillow, requests, tqdm)
2. **Download dataset** - Fetch MNIST handwritten digits dataset from fastai's collection (60k training images, 10k test images)
3. **Build model** - Create a simple 2-layer neural network (784 input → 128 hidden → 10 output) using tinygrad's API
4. **Train model** - Run training for 5 epochs with SGD optimizer, batch size 128, learning rate 0.01
5. **Monitor results** - Track loss per epoch, validation accuracy during training, and final test accuracy
6. **Check GPU usage** - Verify Intel GPU detection and compute runtime availability

**Key features demonstrated:**
- Remote Python execution with virtual environments using `%%vms python:venv_name persistent file.py`
- Tinygrad's tensor operations, automatic differentiation, and optimizer with Intel GPU 
- Training loop with batching, forward/backward passes, and metrics logging 


---


In [ ]:
# Create venv with tinygrad and necessary packages
setup_venv(
    venv_name='tinygrad_env', 
    packages=['tinygrad', 'numpy', 'pillow', 'requests', 'tqdm', 'kagglehub'],
    force_reinstall=False
)


Setting up virtual environment: tinygrad_env

2. Checking for existing virtual environment...


   ✓ Virtual environment already exists: tinygrad_env

3. Upgrading pip...


   ✓ Pip upgraded

4. Installing packages...
   Installing: tinygrad numpy pillow requests tqdm kagglehub


     Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
     Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)


   Using cached packaging-25.0-py3-none-any.whl (66 kB)
   Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl (807 kB)


   

5. Verifying installation...



   Installed packages:
kagglehub                0.3.13
numpy                    2.3.4
pillow                   12.0.0
requests                 2.32.5
tinygrad                 0.11.0
tqdm                     4.67.1


✓ Virtual environment setup complete!

Usage:
   %%vms python:tinygrad_env
   %%vms python:tinygrad_env persistent script.py


In [ ]:
%%vms python:tinygrad_env 

import kagglehub

# Download latest version
path = kagglehub.dataset_download("hojjatk/mnist-dataset")

print("Path to dataset files:", path)

100%|██████████| 22.0M/22.0M [00:06<00:00, 3.46MB/s]

Extracting files...
Path to dataset files: /home/solvituser/.cache/kagglehub/datasets/hojjatk/mnist-dataset/versions/1



In [ ]:
%%vms python:tinygrad_env 

import kagglehub
# Download to cache
cache_path = kagglehub.dataset_download("hojjatk/mnist-dataset")
print("Downloaded to cache:", cache_path)


Downloaded to cache: /home/solvituser/.cache/kagglehub/datasets/hojjatk/mnist-dataset/versions/1



In [ ]:
%%vms

ls -lah /home/solvituser/.cache/kagglehub/datasets/hojjatk/mnist-dataset/versions/1

total 53M
drwxrwxr-x 6 solvituser solvituser 4.0K Oct 15 22:58 .
drwxrwxr-x 3 solvituser solvituser 4.0K Oct 15 22:58 ..
drwxrwxr-x 2 solvituser solvituser 4.0K Oct 15 22:58 t10k-images-idx3-ubyte
-rw-rw-r-- 1 solvituser solvituser 7.5M Oct 15 22:58 t10k-images.idx3-ubyte
drwxrwxr-x 2 solvituser solvituser 4.0K Oct 15 22:58 t10k-labels-idx1-ubyte
-rw-rw-r-- 1 solvituser solvituser 9.8K Oct 15 22:58 t10k-labels.idx1-ubyte
drwxrwxr-x 2 solvituser solvituser 4.0K Oct 15 22:58 train-images-idx3-ubyte
-rw-rw-r-- 1 solvituser solvituser  45M Oct 15 22:58 train-images.idx3-ubyte
drwxrwxr-x 2 solvituser solvituser 4.0K Oct 15 22:58 train-labels-idx1-ubyte
-rw-rw-r-- 1 solvituser solvituser  59K Oct 15 22:58 train-labels.idx1-ubyte



In [ ]:
%%vms python:tinygrad_env persistent mnist_loader.py

import struct
import numpy as np
from pathlib import Path

def load_idx_images(filename):
    """Load images from IDX file format"""
    with open(filename, 'rb') as f:
        # Read magic number and dimensions
        magic = struct.unpack('>I', f.read(4))[0]
        num_images = struct.unpack('>I', f.read(4))[0]
        rows = struct.unpack('>I', f.read(4))[0]
        cols = struct.unpack('>I', f.read(4))[0]
        
        # Read image data
        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(num_images, rows, cols)
        
        return images

def load_idx_labels(filename):
    """Load labels from IDX file format"""
    with open(filename, 'rb') as f:
        # Read magic number and count
        magic = struct.unpack('>I', f.read(4))[0]
        num_labels = struct.unpack('>I', f.read(4))[0]
        
        # Read label data
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        
        return labels

# Path to dataset
data_path = Path('/home/solvituser/.cache/kagglehub/datasets/hojjatk/mnist-dataset/versions/1')

# Load training data
print("Loading training data...")
train_images = load_idx_images(data_path / 'train-images.idx3-ubyte')
train_labels = load_idx_labels(data_path / 'train-labels.idx1-ubyte')
print(f"  Train images shape: {train_images.shape}")
print(f"  Train labels shape: {train_labels.shape}")

# Load test data
print("\nLoading test data...")
test_images = load_idx_images(data_path / 't10k-images.idx3-ubyte')
test_labels = load_idx_labels(data_path / 't10k-labels.idx1-ubyte')
print(f"  Test images shape: {test_images.shape}")
print(f"  Test labels shape: {test_labels.shape}")

# Save as numpy arrays
print("\nSaving data as numpy arrays...")
np.save('train_images.npy', train_images)
np.save('train_labels.npy', train_labels)
np.save('test_images.npy', test_images)
np.save('test_labels.npy', test_labels)

print("\n✓ Data loaded and saved successfully!")
print("\nData summary:")
print(f"  Training samples: {len(train_images)}")
print(f"  Test samples: {len(test_images)}")
print(f"  Image size: {train_images.shape[1]}x{train_images.shape[2]}")
print(f"  Number of classes: {len(np.unique(train_labels))}")

Loading training data...
  Train images shape: (60000, 28, 28)
  Train labels shape: (60000,)

Loading test data...
  Test images shape: (10000, 28, 28)
  Test labels shape: (10000,)

Saving data as numpy arrays...

✓ Data loaded and saved successfully!

Data summary:
  Training samples: 60000
  Test samples: 10000
  Image size: 28x28
  Number of classes: 10



In [ ]:
%%vms python:tinygrad_env

from tinygrad import Device

print(f"Current device: {Device.DEFAULT}") #make sure user has access

Current device: GPU



In [ ]:
%%vms python:tinygrad_env persistent train_mnist.py

import pickle
import gzip
import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import Linear
from tinygrad.nn.optim import SGD
from pathlib import Path
import time

print("=" * 60)
print("MNIST Training with Tinygrad on Intel GPU")
print("=" * 60)

# Load data
data_path = Path('mnist_data')
with gzip.open(data_path / 'mnist.pkl.gz', 'rb') as f:
    train_data, val_data, test_data = pickle.load(f, encoding='latin1')

X_train, y_train = train_data
X_val, y_val = val_data

# Use smaller subset for quick demo
n_samples = 5000
X_train, y_train = X_train[:n_samples], y_train[:n_samples]
X_val, y_val = X_val[:1000], y_val[:1000]

print(f"\nDataset loaded:")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")

# Simple 2-layer neural network
class SimpleNet:
    def __init__(self):
        self.l1 = Linear(784, 128)
        self.l2 = Linear(128, 10)
    
    def __call__(self, x):
        x = self.l1(x).relu()
        return self.l2(x)

model = SimpleNet()
opt = SGD([model.l1.weight, model.l1.bias, model.l2.weight, model.l2.bias], lr=0.01)

print(f"\nModel architecture:")
print(f"  Input: 784 -> Hidden: 128 -> Output: 10")
print(f"  Optimizer: SGD (lr=0.01)")

# Training loop
batch_size = 128
epochs = 5

print(f"\n{'='*60}")
print("Starting training...")
print(f"{'='*60}\n")

for epoch in range(epochs):
    epoch_start = time.time()
    epoch_loss = 0
    n_batches = 0
    
    # Shuffle data
    indices = np.random.permutation(len(X_train))
    Tensor.training=True    
    for i in range(0, len(X_train), batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = Tensor(X_train[batch_idx])
        y_batch = Tensor(y_train[batch_idx])
        
        # Forward pass
        logits = model(X_batch)
        
        # Compute loss using one-hot encoding
        y_onehot = Tensor(np.eye(10)[y_train[batch_idx]])
        log_probs = logits.log_softmax(axis=1)
        loss = -(y_onehot * log_probs).sum() / len(batch_idx)
        
        # Backward pass
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        epoch_loss += loss.numpy()
        n_batches += 1
    
    # Validation accuracy
    val_logits = model(Tensor(X_val))
    val_preds = val_logits.numpy().argmax(axis=1)
    val_acc = (val_preds == y_val).mean()
    
    epoch_time = time.time() - epoch_start
    avg_loss = epoch_loss / n_batches
    
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Time: {epoch_time:.2f}s")

print(f"\n{'='*60}")
print("Training complete!")
print(f"{'='*60}")
Tensor.training=False
# Final test accuracy
test_logits = model(Tensor(test_data[0]))
test_preds = test_logits.numpy().argmax(axis=1)
test_acc = (test_preds == test_data[1]).mean()
print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")


STDERR: Traceback (most recent call last):
  File "/home/solvituser/train_mnist.py", line 17, in <module>
    with gzip.open(data_path / 'mnist.pkl.gz', 'rb') as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/gzip.py", line 61, in open
    binary_file = GzipFile(filename, gz_mode, compresslevel)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/gzip.py", line 192, in __init__
    fileobj = self.myfileobj = builtins.open(filename, mode or 'rb')
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'mnist_data/mnist.pkl.gz'

MNIST Training with Tinygrad on Intel GPU



ok let's go through the error carefully to identify

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at the error, the issue is clear:

```
FileNotFoundError: [Errno 2] No such file or directory: 'mnist_data/mnist.pkl.gz'
```

The script is trying to load data from `mnist_data/mnist.pkl.gz`, but this file doesn't exist. 

However, you already have the MNIST data! Look back at your earlier code - you successfully loaded the data from the Kaggle dataset and saved it as numpy arrays:
- `train_images.npy`
- `train_labels.npy`
- `test_images.npy`
- `test_labels.npy`

So the fix is to update the `train_mnist.py` script to load from these `.npy` files instead of trying to load from `mnist.pkl.gz`.

Would you like to modify the script to use the numpy files you already have?

ok give modified version

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

```python
%%vms

# First, let's clear the old persistent file
rm -f train_mnist.py
```

Then run this updated version:

```python
%%vms python:tinygrad_env persistent train_mnist.py

import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import Linear
from tinygrad.nn.optim import SGD
import time

print("=" * 60)
print("MNIST Training with Tinygrad on Intel GPU")
print("=" * 60)

# Load data from numpy files
print("\nLoading data from numpy files...")
X_train = np.load('train_images.npy')
y_train = np.load('train_labels.npy')
X_test = np.load('test_images.npy')
y_test = np.load('test_labels.npy')

# Flatten images and normalize
X_train = X_train.reshape(-1, 784).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, 784).astype(np.float32) / 255.0

# Create validation split
n_val = 10000
X_val, y_val = X_train[-n_val:], y_train[-n_val:]
X_train, y_train = X_train[:-n_val], y_train[:-n_val]

# Use smaller subset for quick demo
n_samples = 5000
X_train, y_train = X_train[:n_samples], y_train[:n_samples]
X_val, y_val = X_val[:1000], y_val[:1000]

print(f"\nDataset loaded:")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")

# Simple 2-layer neural network
class SimpleNet:
    def __init__(self):
        self.l1 = Linear(784, 128)
        self.l2 = Linear(128, 10)
    
    def __call__(self, x):
        x = self.l1(x).relu()
        return self.l2(x)

model = SimpleNet()
opt = SGD([model.l1.weight, model.l1.bias, model.l2.weight, model.l2.bias], lr=0.01)

print(f"\nModel architecture:")
print(f"  Input: 784 -> Hidden: 128 -> Output: 10")
print(f"  Optimizer: SGD (lr=0.01)")

# Training loop
batch_size = 128
epochs = 5

print(f"\n{'='*60}")
print("Starting training...")
print(f"{'='*60}\n")

for epoch in range(epochs):
    epoch_start = time.time()
    epoch_loss = 0
    n_batches = 0
    
    # Shuffle data
    indices = np.random.permutation(len(X_train))
    Tensor.training = True
    
    for i in range(0, len(X_train), batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = Tensor(X_train[batch_idx])
        y_batch = y_train[batch_idx]
        
        # Forward pass
        logits = model(X_batch)
        
        # Compute loss using one-hot encoding
        y_onehot = Tensor(np.eye(10)[y_batch])
        log_probs = logits.log_softmax(axis=1)
        loss = -(y_onehot * log_probs).sum() / len(batch_idx)
        
        # Backward pass
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        epoch_loss += loss.numpy()
        n_batches += 1
    
    # Validation accuracy
    Tensor.training = False
    val_logits = model(Tensor(X_val))
    val_preds = val_logits.numpy().argmax(axis=1)
    val_acc = (val_preds == y_val).mean()
    
    epoch_time = time.time() - epoch_start
    avg_loss = epoch_loss / n_batches
    
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Time: {epoch_time:.2f}s")

print(f"\n{'='*60}")
print("Training complete!")
print(f"{'='*60}")

# Final test accuracy
Tensor.training = False
test_logits = model(Tensor(X_test))
test_preds = test_logits.numpy().argmax(axis=1)
test_acc = (test_preds == y_test).mean()
print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")
```

Key changes:
1. Removed the `pickle.load()` code that was looking for `mnist.pkl.gz`
2. Load directly from your `.npy` files
3. Flatten and normalize the images (reshape to 784 pixels, divide by 255)
4. Create validation split from the training data
5. Fixed the `Tensor.training` flag placement

In [ ]:
%%vms

# First, let's clear the old persistent file
rm -f train_mnist.py

In [ ]:
%%vms python:tinygrad_env persistent train_mnist.py

import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import Linear
from tinygrad.nn.optim import SGD
import time

print("=" * 60)
print("MNIST Training with Tinygrad on Intel GPU")
print("=" * 60)

# Load data from numpy files
print("\nLoading data from numpy files...")
X_train = np.load('train_images.npy')
y_train = np.load('train_labels.npy')
X_test = np.load('test_images.npy')
y_test = np.load('test_labels.npy')

# Flatten images and normalize
X_train = X_train.reshape(-1, 784).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, 784).astype(np.float32) / 255.0

# Create validation split
n_val = 10000
X_val, y_val = X_train[-n_val:], y_train[-n_val:]
X_train, y_train = X_train[:-n_val], y_train[:-n_val]

# Use smaller subset for quick demo
n_samples = 5000
X_train, y_train = X_train[:n_samples], y_train[:n_samples]
X_val, y_val = X_val[:1000], y_val[:1000]

print(f"\nDataset loaded:")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")

# Simple 2-layer neural network
class SimpleNet:
    def __init__(self):
        self.l1 = Linear(784, 128)
        self.l2 = Linear(128, 10)
    
    def __call__(self, x):
        x = self.l1(x).relu()
        return self.l2(x)

model = SimpleNet()
opt = SGD([model.l1.weight, model.l1.bias, model.l2.weight, model.l2.bias], lr=0.01)

print(f"\nModel architecture:")
print(f"  Input: 784 -> Hidden: 128 -> Output: 10")
print(f"  Optimizer: SGD (lr=0.01)")

# Training loop
batch_size = 128
epochs = 5

print(f"\n{'='*60}")
print("Starting training...")
print(f"{'='*60}\n")

for epoch in range(epochs):
    epoch_start = time.time()
    epoch_loss = 0
    n_batches = 0
    
    # Shuffle data
    indices = np.random.permutation(len(X_train))
    Tensor.training = True
    
    for i in range(0, len(X_train), batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = Tensor(X_train[batch_idx])
        y_batch = y_train[batch_idx]
        
        # Forward pass
        logits = model(X_batch)
        
        # Compute loss using one-hot encoding
        y_onehot = Tensor(np.eye(10)[y_batch])
        log_probs = logits.log_softmax(axis=1)
        loss = -(y_onehot * log_probs).sum() / len(batch_idx)
        
        # Backward pass
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        epoch_loss += loss.numpy()
        n_batches += 1
    
    # Validation accuracy
    Tensor.training = False
    val_logits = model(Tensor(X_val))
    val_preds = val_logits.numpy().argmax(axis=1)
    val_acc = (val_preds == y_val).mean()
    
    epoch_time = time.time() - epoch_start
    avg_loss = epoch_loss / n_batches
    
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Time: {epoch_time:.2f}s")

print(f"\n{'='*60}")
print("Training complete!")
print(f"{'='*60}")

# Final test accuracy
Tensor.training = False
test_logits = model(Tensor(X_test))
test_preds = test_logits.numpy().argmax(axis=1)
test_acc = (test_preds == y_test).mean()
print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")

STDERR: Traceback (most recent call last):
  File "/home/solvituser/train_mnist.py", line 88, in <module>
    opt.step()
  File "/home/solvituser/tinygrad_env/lib/python3.12/site-packages/tinygrad/nn/optim.py", line 41, in step
    Tensor.realize(*self.schedule_step())
  File "/home/solvituser/tinygrad_env/lib/python3.12/site-packages/tinygrad/tensor.py", line 4432, in _wrapper
    ret = fn(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/home/solvituser/tinygrad_env/lib/python3.12/site-packages/tinygrad/tensor.py", line 266, in realize
    run_schedule(*self.schedule_with_vars(*lst), do_update_stats=do_update_stats)
  File "/home/solvituser/tinygrad_env/lib/python3.12/site-packages/tinygrad/engine/realize.py", line 195, in run_schedule
    for si, ei in lower_schedule(schedule):
  File "/home/solvituser/tinygrad_env/lib/python3.12/site-packages/tinygrad/engine/realize.py", line 188, in lower_schedule
    raise e
  File "/home/solvituser/tinygrad_env/lib/python3.12/site-packages


ok what happened let's go through the error message

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at the error message carefully:

**Main Error:**
```
tinygrad.device.CompileError: OpenCL Compile Error
error: use of type 'double' requires cl_khr_fp64 support
```

**What's happening:**

1. **The Intel GPU doesn't support double precision (float64)** - The OpenCL compiler is trying to use `double` types, but your Intel GPU doesn't have the `cl_khr_fp64` extension enabled/available.

2. **Where doubles are coming from** - Look at this line in the error:
   ```
   __global double* data1_1280
   ```
   Tinygrad is generating code that uses `double` for one of the buffers.

3. **The specific operation** - The error shows it's happening during the optimizer step (`opt.step()`), and the values being used are things like `-0.0078125` which is `1/128` (the learning rate divided by batch size).

**The root cause:** When we created the labels, they're probably being stored as `int64` (which numpy uses by default), and somewhere in the computation this is getting converted to `float64` instead of `float32`.

**The fix:** We need to ensure ALL data is `float32`, including the labels when we create the one-hot encoding. Let me show you the corrected version:

```python
%%vms

rm -f train_mnist.py
```

```python
%%vms python:tinygrad_env persistent train_mnist.py

import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import Linear
from tinygrad.nn.optim import SGD
import time

print("=" * 60)
print("MNIST Training with Tinygrad on Intel GPU")
print("=" * 60)

# Load data from numpy files
print("\nLoading data from numpy files...")
X_train = np.load('train_images.npy')
y_train = np.load('train_labels.npy')
X_test = np.load('test_images.npy')
y_test = np.load('test_labels.npy')

# Flatten images and normalize - ENSURE FLOAT32
X_train = X_train.reshape(-1, 784).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, 784).astype(np.float32) / 255.0

# Create validation split
n_val = 10000
X_val, y_val = X_train[-n_val:], y_train[-n_val:]
X_train, y_train = X_train[:-n_val], y_train[:-n_val]

# Use smaller subset for quick demo
n_samples = 5000
X_train, y_train = X_train[:n_samples], y_train[:n_samples]
X_val, y_val = X_val[:1000], y_val[:1000]

print(f"\nDataset loaded:")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")

# Simple 2-layer neural network
class SimpleNet:
    def __init__(self):
        self.l1 = Linear(784, 128)
        self.l2 = Linear(128, 10)
    
    def __call__(self, x):
        x = self.l1(x).relu()
        return self.l2(x)

model = SimpleNet()
opt = SGD([model.l1.weight, model.l1.bias, model.l2.weight, model.l2.bias], lr=0.01)

print(f"\nModel architecture:")
print(f"  Input: 784 -> Hidden: 128 -> Output: 10")
print(f"  Optimizer: SGD (lr=0.01)")

# Training loop
batch_size = 128
epochs = 5

print(f"\n{'='*60}")
print("Starting training...")
print(f"{'='*60}\n")

for epoch in range(epochs):
    epoch_start = time.time()
    epoch_loss = 0
    n_batches = 0
    
    # Shuffle data
    indices = np.random.permutation(len(X_train))
    Tensor.training = True
    
    for i in range(0, len(X_train), batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = Tensor(X_train[batch_idx])
        y_batch = y_train[batch_idx]
        
        # Forward pass
        logits = model(X_batch)
        
        # Compute loss - ENSURE ONE-HOT IS FLOAT32
        y_onehot = Tensor(np.eye(10, dtype=np.float32)[y_batch])
        log_probs = logits.log_softmax(axis=1)
        loss = -(y_onehot * log_probs).sum() / len(batch_idx)
        
        # Backward pass
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        epoch_loss += loss.numpy()
        n_batches += 1
    
    # Validation accuracy
    Tensor.training = False
    val_logits = model(Tensor(X_val))
    val_preds = val_logits.numpy().argmax(axis=1)
    val_acc = (val_preds == y_val).mean()
    
    epoch_time = time.time() - epoch_start
    avg_loss = epoch_loss / n_batches
    
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Time: {epoch_time:.2f}s")

print(f"\n{'='*60}")
print("Training complete!")
print(f"{'='*60}")

# Final test accuracy
Tensor.training = False
test_logits = model(Tensor(X_test))
test_preds = test_logits.numpy().argmax(axis=1)
test_acc = (test_preds == y_test).mean()
print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")
```

**Key change:** Line 71 now explicitly specifies `dtype=np.float32`:
```python
y_onehot = Tensor(np.eye(10, dtype=np.float32)[y_batch])
```

This ensures the one-hot encoded labels are float32, preventing tinygrad from trying to use double precision on your Intel GPU.

In [ ]:
%%vms

rm -f train_mnist.py

In [ ]:
%%vms python:tinygrad_env persistent train_mnist.py

import numpy as np
from tinygrad.tensor import Tensor
from tinygrad.nn import Linear
from tinygrad.nn.optim import SGD
import time

print("=" * 60)
print("MNIST Training with Tinygrad on Intel GPU")
print("=" * 60)

# Load data from numpy files
print("\nLoading data from numpy files...")
X_train = np.load('train_images.npy')
y_train = np.load('train_labels.npy')
X_test = np.load('test_images.npy')
y_test = np.load('test_labels.npy')

# Flatten images and normalize - ENSURE FLOAT32
X_train = X_train.reshape(-1, 784).astype(np.float32) / 255.0
X_test = X_test.reshape(-1, 784).astype(np.float32) / 255.0

# Create validation split
n_val = 10000
X_val, y_val = X_train[-n_val:], y_train[-n_val:]
X_train, y_train = X_train[:-n_val], y_train[:-n_val]

# Use smaller subset for quick demo
n_samples = 5000
X_train, y_train = X_train[:n_samples], y_train[:n_samples]
X_val, y_val = X_val[:1000], y_val[:1000]

print(f"\nDataset loaded:")
print(f"  Training: {len(X_train)} samples")
print(f"  Validation: {len(X_val)} samples")
print(f"  Test: {len(X_test)} samples")

# Simple 2-layer neural network
class SimpleNet:
    def __init__(self):
        self.l1 = Linear(784, 128)
        self.l2 = Linear(128, 10)
    
    def __call__(self, x):
        x = self.l1(x).relu()
        return self.l2(x)

model = SimpleNet()
opt = SGD([model.l1.weight, model.l1.bias, model.l2.weight, model.l2.bias], lr=0.01)

print(f"\nModel architecture:")
print(f"  Input: 784 -> Hidden: 128 -> Output: 10")
print(f"  Optimizer: SGD (lr=0.01)")

# Training loop
batch_size = 128
epochs = 5

print(f"\n{'='*60}")
print("Starting training...")
print(f"{'='*60}\n")

for epoch in range(epochs):
    epoch_start = time.time()
    epoch_loss = 0
    n_batches = 0
    
    # Shuffle data
    indices = np.random.permutation(len(X_train))
    Tensor.training = True
    
    for i in range(0, len(X_train), batch_size):
        batch_idx = indices[i:i+batch_size]
        X_batch = Tensor(X_train[batch_idx])
        y_batch = y_train[batch_idx]
        
        # Forward pass
        logits = model(X_batch)
        
        # Compute loss - ENSURE ONE-HOT IS FLOAT32
        y_onehot = Tensor(np.eye(10, dtype=np.float32)[y_batch])
        log_probs = logits.log_softmax(axis=1)
        loss = -(y_onehot * log_probs).sum() / len(batch_idx)
        
        # Backward pass
        opt.zero_grad()
        loss.backward()
        opt.step()
        
        epoch_loss += loss.numpy()
        n_batches += 1
    
    # Validation accuracy
    Tensor.training = False
    val_logits = model(Tensor(X_val))
    val_preds = val_logits.numpy().argmax(axis=1)
    val_acc = (val_preds == y_val).mean()
    
    epoch_time = time.time() - epoch_start
    avg_loss = epoch_loss / n_batches
    
    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Val Acc: {val_acc*100:.2f}% | Time: {epoch_time:.2f}s")

print(f"\n{'='*60}")
print("Training complete!")
print(f"{'='*60}")

# Final test accuracy
Tensor.training = False
test_logits = model(Tensor(X_test))
test_preds = test_logits.numpy().argmax(axis=1)
test_acc = (test_preds == y_test).mean()
print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")

MNIST Training with Tinygrad on Intel GPU

Loading data from numpy files...

Dataset loaded:
  Training: 5000 samples
  Validation: 1000 samples
  Test: 10000 samples

Model architecture:
  Input: 784 -> Hidden: 128 -> Output: 10
  Optimizer: SGD (lr=0.01)

Starting training...

Epoch 1/5 | Loss: 2.2564 | Val Acc: 30.50% | Time: 10.43s
Epoch 2/5 | Loss: 2.1668 | Val Acc: 51.40% | Time: 1.35s
Epoch 3/5 | Loss: 2.0657 | Val Acc: 62.70% | Time: 1.41s
Epoch 4/5 | Loss: 1.9440 | Val Acc: 65.00% | Time: 1.29s
Epoch 5/5 | Loss: 1.8175 | Val Acc: 69.40% | Time: 1.36s

Training complete!

Final Test Accuracy: 71.37%

